<a href="https://colab.research.google.com/github/a-mhamdi/jlai/blob/main/Codes/Julia/Part-3/transfer_learning_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning (MNIST DATASET)
---

In [ ]:
versioninfo()  # -> 1.11.5

In [ ]:
pkgs = """[deps]
BSON = "fbb218c0-5317-5bc6-957e-2ee96dd4b1f0"
CUDA = "052768ef-5323-5732-b1bb-66c8b64840ba"
CairoMakie = "13f3f980-e62b-5c42-98c6-ff1f3baf88f0"
DataAugmentation = "88a5189c-e7ff-4f85-ac6b-e6158070f02e"
Flux = "587475ba-b771-5e3f-ad9e-33799f191a9c"
ImageInTerminal = "d8c32880-2388-543b-8c61-d9f865259254"
ImageShow = "4e3cecfd-b093-5904-9786-8bbb286a6a31"
Images = "916415d5-f1e6-5110-898d-aaa5f9f070e0"
JLD2 = "033835bb-8acc-5ee8-8aae-3f567f8a3819"
MLDatasets = "eb30cadb-4394-5ae3-aed4-317e484a6458"
Metalhead = "dbeba491-748d-5e0e-a39e-b530a07fa0cc"
ProgressMeter = "92933f4c-e287-5a05-a399-4b506db050ca"
Statistics = "10745b16-79ce-11e8-11f9-7d13ad32a3b2"
cuDNN = "02a925ec-e4fe-4b08-9a7e-0d78e3d38ccd"
"""

open("Project.toml", "w") do file
    write(file, pkgs)
end

In [ ]:
_ = begin
  import Pkg;
  Pkg.activate(".");
  Pkg.instantiate();
end

In [ ]:
Pkg.status()

Load the pre-trained model

In [ ]:
using CUDA
using Metalhead

[API Reference](https://fluxml.ai/Metalhead.jl/dev/api/reference/#API-Reference)

In [ ]:
vgg = VGG(16; pretrain=true)

In [ ]:
using DataAugmentation

In [ ]:
tfm = CenterCrop((224, 224)) |> ImageToTensor()

In [ ]:
using Flux
using Flux: onecold, onehotbatch

In [ ]:
model = Chain(
    vgg[1:end-1],
    vgg[end][1:end-1],
    # Replace the last layer
    Dense(4096, 256),
    Dense(256, 10)
    ) |> gpu

In [ ]:
using MLDatasets

Load the MNIST dataset

In [ ]:
function get_data(split)
    data = MNIST(split)
    imgs, y = data.features ./ 255, onehotbatch(data.targets, 0:9);
    X = []
    for i in 1:length(y)
        img = apply(tfm, Image(RGB.(imgs[:,:,i]))) |> itemdata
        push!(X, img)
    end
    loader = Flux.Data.DataLoader((X, y); batchsize=64, shuffle=true) |> gpu;
    return loader
end

In [ ]:
using Images

In [ ]:
train_loader = get_data(:train);
test_loader = get_data(:test);

**Define a loss function and an optimizer**

In [ ]:
loss_fn = Flux.logitcrossentropy
opt_state = Flux.setup(Adam(3e-3), model[end]) # Freeze the weights of the pre-trained layers

In [ ]:
using ProgressMeter

In [ ]:
epochs = 3

Fine-tune the model

In [ ]:
for epoch in 1:epochs

    @showprogress for (X, y) in train_loader
        # Compute the gradient of the loss with respect to the model's parameters
        loss, ∇ = Flux.withgradient(model) do m
            ŷ = m(X)
            loss_fn(ŷ, y)
        end
        # Update the model's parameters using the optimizer
        Flux.update!(opt_state, model, ∇[1])
    end

    @info "Calculate the accuracy on the test set"
    for (X, y) in test_loader
        ys = onecold(y, 0:9) |> cpu
        ŷ  = onecold(model(X), 0:9) |> cpu
        accuracy = sum( ys .== ŷ ) / length(ys)
        println("Epoch: $epoch, Accuracy: $accuracy")
    end

end